In [66]:
# service test

In [67]:
__author__ = 'Kushal'

from datetime import datetime
from dateutil import relativedelta
import statistics as stat
import numpy as np
import json
# from app.main import logger

def excl_none_operation(li, func):
    lis = [i for i in li if i is not None]
    if func == 'sum':
        return sum(lis) if len(lis) > 0 else -99999
    elif func == 'max':
        return max(lis) if len(lis) > 0 else -99999
    elif func == 'min':
        return min(lis) if len(lis) > 0 else -99999
    elif func == 'count_non_zero':
        return np.count_nonzero(lis) if len(lis) > 0 else -99999
    elif func == 'mean':
        return np.mean(lis) if len(lis) > 0 else -99999


def date_format_payment_inq(d):
    if d == '00000000' or d == '' or d is None:
        d = '01-01-1900'
    return datetime.strptime(d, '%Y-%m-%d')

def getinquiry_list(d):
    data = d['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']
    enqdata_list = data['CREDIT_INQUIRY'] if 'CREDIT_INQUIRY' in data else None
    if type(enqdata_list) == list:
        enqdata = data['CREDIT_INQUIRY'] if 'CREDIT_INQUIRY' in data else None
#     elif type(enqdata_list) != list:
#         enqdata = []
#         enqdata.append(enqdata_list)
    else:
        enqdata = []
    return enqdata

def rollup_inquiry(input_data, reqDate):
    # inquiry_data = getinquiry_list(input_data)
    inquiry_data = input_data
    if inquiry_data:
        inquiry = [{'ec_in01_inquiry_date': d['_Date'], 'ApplicationDateRequested_new': reqDate} for d in inquiry_data]
        inquiry = [{**d, 'DATEREPORTED_new': date_format_payment_inq(d['ec_in01_inquiry_date'])} for d in inquiry]
        L1M = reqDate - relativedelta.relativedelta(months=1)
        inquiry = [{**d, 'inquiries_L1m': d['ApplicationDateRequested_new'] - relativedelta.relativedelta(months=1),
                    'inquiries_L2m': d['ApplicationDateRequested_new'] - relativedelta.relativedelta(months=2),
                    'inquiries_L3m': d['ApplicationDateRequested_new'] - relativedelta.relativedelta(months=3),
                    'inquiries_L6m': d['ApplicationDateRequested_new'] - relativedelta.relativedelta(months=6),
                    'inquiries_L12m': d['ApplicationDateRequested_new'] - relativedelta.relativedelta(months=12),
                    'inquiries_L24m': d['ApplicationDateRequested_new'] - relativedelta.relativedelta(months=24)} for d in inquiry]
        inquiry = [{**d, 'days_diff_disb_To_enqDate' : (d['ApplicationDateRequested_new'] - d['DATEREPORTED_new']).days} for d in inquiry]
        inquiry = [{**d, 'recent_inq_inMnths': round(d['days_diff_disb_To_enqDate']/30)} for d in inquiry]
        inquiry_rollup = {'tot_enq': len(inquiry_data),
        'enq_L1m': sum([1 for item in inquiry if (item['DATEREPORTED_new'] >= item['inquiries_L1m'])]),
        'enq_L2m': sum([1 for item in inquiry if (item['DATEREPORTED_new'] >= item['inquiries_L2m'])]),
        'enq_L3m': sum([1 for item in inquiry if (item['DATEREPORTED_new'] >= item['inquiries_L3m'])]),
        'enq_L6m': sum([1 for item in inquiry if (item['DATEREPORTED_new'] >= item['inquiries_L6m'])]),
        'enq_L12m': sum([1 for item in inquiry if (item['DATEREPORTED_new'] >= item['inquiries_L12m'])]),
        'enq_L24m': sum([1 for item in inquiry if (item['DATEREPORTED_new'] >= item['inquiries_L24m'])]),
        'time_since_oldest_enquiry': excl_none_operation([d['recent_inq_inMnths'] for d in inquiry], 'max')}
        # Enquiry Adjustments (post 07/01):
        # commented on 27/01
        # if reqDate >= datetime.strptime('2021-07-01', '%Y-%m-%d'):
        #     for key, value in inquiry_rollup.items():
        #         if key != 'time_since_oldest_enquiry':
        #             inquiry_rollup[key] = inquiry_rollup[key] - 1
        return inquiry_rollup
    else:
        inquiry_rollup = {'tot_enq': -99999,
        'enq_L1m': -99999,
        'enq_L2m': -99999,
        'enq_L3m': -99999,
        'enq_L6m': -99999,
        'enq_L12m': -99999,
        'enq_L24m': -99999,
        'time_since_oldest_enquiry': -99999}
        return inquiry_rollup

def getcollection_list(input_data):
    data = input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['_DATA_SET']
    for row in data:
        if row['_Name']=='_TotalCollections':
            _OldestTradeDate = datetime_object = datetime.strptime(row['_Value'], '%y-%m')
    collection_list = data['collection'] if 'collection' in data else None
    if type(collection_list) == list:
        collection_data = data['collection'] if 'collection' in data else None
#     elif type(collection_list) != list:
#         collection_data = []
#         collection_data.append(collection_list)
    else:
        collection_data = []
    return collection_data

def rollup_collection(input_data, reqDate):
    collection_rollup = {'times_collections_ever':0, 'collection_not_paid_ever':0,
                         'collection_paid_ever':0, 'ttl_collections_amount':0}
    data = input_data
    
    # data = input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_SUMMARY']['_DATA_SET']
    num_collections = 0
    
    for row in data:
        if row['_Name']=='_TotalCollections':
            num_collections = row['_Value']              
            break

    if num_collections >= 0:
        collection_rollup['times_collections_ever_count'] = num_collections
        collection_rollup['collection_not_paid_ever_count'] = -99999
        collection_rollup['collection_paid_ever_count'] = -99999
        collection_rollup['ttl_collections_amount'] = -99999
    elif num_collections<0:
        collection_rollup['times_collections_ever_count'] = -99999
        collection_rollup['collection_not_paid_ever_count'] = -99999
        collection_rollup['collection_paid_ever_count'] = -99999
        collection_rollup['ttl_collections_amount'] = -99999
    return collection_rollup

def rollup_bankruptcy(input_data, reqDate):
    bankruptcy_rollup = {'Count_BK_L6M':0, 'Count_BK_L12M':0, 'Count_BK_L24M':0, 'Count_BK_L60M':0,
                         'Count_Individual_BK_L6M':0, 'Count_Individual_BK_L12M':0, 'Count_Individual_BK_L24M':0, 'Count_Individual_BK_L60M':0,
                         'Count_Joint_BK_L6M':0, 'Count_Joint_BK_L12M':0, 'Count_Joint_BK_L24M':0, 'Count_Joint_BK_L60M':0}    
    
    
    # data = input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_SUMMARY']['_DATA_SET']
    data = input_data
    num_months_since_bk = 0
    
    for row in data:
        if row['_Name']=='_NumberOfMonthsSinceMostRecentBankruptcy':
            num_months_since_bk = row['_Value']
            break
            
    if num_months_since_bk > 0:
        if num_months_since_bk <= 6:
            bankruptcy_rollup['Count_BK_L6M'] = num_months_since_bk
            bankruptcy_rollup['Count_Individual_BK_L6M'] = -99999
            bankruptcy_rollup['Count_Joint_BK_L6M'] = -99999
        elif num_months_since_bk <= 12:
            bankruptcy_rollup['Count_BK_L12M'] = num_months_since_bk
            bankruptcy_rollup['Count_Individual_BK_L12M'] = -99999
            bankruptcy_rollup['Count_Joint_BK_L12M'] = -99999
        elif num_months_since_bk <= 24:
            bankruptcy_rollup['Count_BK_L24M'] = num_months_since_bk
            bankruptcy_rollup['Count_Individual_BK_L24M'] = -99999
            bankruptcy_rollup['Count_Joint_BK_L24M'] = -99999
        elif num_months_since_bk <= 60:
            bankruptcy_rollup['Count_BK_L60M'] = num_months_since_bk   
            bankruptcy_rollup['Count_Individual_BK_L60M'] = -99999
            bankruptcy_rollup['Count_Joint_BK_L60M'] = -99999
    if num_months_since_bk < 0:
        for key in bankruptcy_rollup.keys():
            bankruptcy_rollup[key] = -99999
    return bankruptcy_rollup

def convert_to_date(str_date):
    return(datetime.strptime(str_date, '%Y-%m'))

def get_split_paygrid_res(li, cond, strind, endind, func, eq):
    resp = None
    if(li != ['']):
        if (eq == 'yes'):
            lis=[x for x in li[strind:endind] if x == cond]
            deflt = 0 if (func == 'sum') else None
            resp = sum(lis) if (len(lis) > 0 and func == 'sum') else max(lis) if (len(lis) > 0 and func == 'max') else len(lis) if func == 'count' else deflt
        else:
            lis=[x for x in li[strind:endind] if x >= cond] if eq else [x for x in li[strind:endind] if x > cond]
            deflt = 0 if (func == 'sum') else None
            resp = sum(lis) if (len(lis) > 0 and func == 'sum') else max(lis) if (len(lis) > 0 and func == 'max') else len(lis) if func == 'count' else deflt  
        return resp
    else:
        return []

def split_grid(s):
    s = str(s)
    return [s[i:i+1] for i in range(0, len(s), 1)]

def open_grid(pat):
    s=list(pat)
    if(len(s) >0):
        s1= [-1 if x=='C' or x== 'N' or x== 'X' or x== 'Z' or x=='O' else int(x) for x in s]
    else:
        s1=['']
    
    return s1

# added on 27/01/2022.. updated 3/02/2022
def res(list_vars,res1a):
    res1a ={}
    for i in list_vars:
        res1a[i] = -99999
        res1a['Bureau_availability'] = 0
        res1a['error_status']='Bureau pull issue or scoring failed due to a technical reason'
    return res1a






# moved from service to utils on 26012022

def mcmc_rollup(oid,app_acc_1, app_inq_1,app_sum_1,bur_list_1,reqdate):
#     logger.info(input)
    reqDate= datetime.now()
    # oid = input_data['id'] if 'id' in input_data else "-99999"

    # reqdate = datetime.strptime(input_data['submitted_datetime'][0:10], '%Y-%m-%d') if 'submitted_datetime' in input_data else datetime.now() 

    main_rollup = { 'Unique_ID':'', 'CR_Score_Type':'', 'CR_Score':0,'error_status':'Bureau pull Success',
                    'Automobile_TL':0, 'CreditCard_TL':0, 'StudentLoan_TL':0, 'Secured_TL':0, 'Unsecured_TL':0, 'ChargeAccount_TL':0,
                    'PersonalLoan_TL':0, 'SecuredCreditCard_TL':0, 'Mortgage_TL':0, 'Installment_TL':0, 'Revolving_TL':0, 'OpenAccount_TL':0,
                    'Num_Open_Automobile_TL':0, 'Num_Open_CreditCard_TL':0, 'Num_Open_StudentLoan_TL':0, 'Num_Open_Secured_TL':0,
                    'Num_Open_Unsecured_TL':0, 'Num_Open_ChargeAccount_TL':0, 'Num_Open_PersonalLoan_TL':0, 'Num_Open_SecuredCreditCard_TL':0,
                    'Num_Open_Mortgage_TL':0, 'Num_Open_Installment_TL':0, 'Num_Open_Revolving_TL':0, 'Num_Open_OpenAccount_TL':0,
                    'Total_TL':0, 'Tot_Active_TL':0, 'Tot_Closed_TL':0, 
                    'Total_TL_Open_L6M':0, 'Tot_Active_TL_L6M':0, 'Total_TL_Open_L12M':0, 'Tot_Active_TL_L12M':0,
                    'Average Age of Accounts':0, 'Age_Oldest_TL_Mnths':0, 'Age_Newest_TL_Mnths':0, 'first_product':'', 'recent_product':'',
                    'Tot_Automobile_Bal':0, 'Tot_Automobile_Limit':0, 'Tot_CreditCard_Bal':0, 'Tot_CreditCard_Limit':0,
                    'Tot_StudentLoan_Bal':0, 'Tot_StudentLoan_Limit':0, 'Tot_SecuredLoan_Bal':0, 'Tot_SecuredLoan_Limit':0,
                    'Tot_UnsecuredLoan_Bal':0, 'Tot_UnsecuredLoan_Limit':0, 'Tot_ChargeLoan_Bal':0, 'Tot_ChargeLoan_Limit':0,
                    'Tot_SecuredCreditCard_Bal':0, 'Tot_SecuredCreditCard_Limit':0, 'Tot_Mortgage_Bal':0, 'Tot_Mortgage_Limit':0,
                    'Tot_Installment_Bal':0, 'Tot_Installment_Limit':0, 'Tot_Revolving_Bal':0, 'Tot_Revolving_Limit':0,
                    'Tot_OpenAccount_Bal':0, 'Tot_OpenAccount_Limit':0,
                    'time_since_first_deliquency':0, 'time_since_recent_deliquency':0,'Bureau_availability': 0,
                    'num_times_delinquent':0, 'max_delinquency_level':0, 'max_recent_deliq':0, 'recent_deliq':0,
                    'num_deliq_6mts':0, 'num_deliq_12mts':0, 'num_deliq_24mts':0, 'num_deliq_6_12mts':0, 'num_tradelines':0,
                    'max_deliq_6mts':0, 'max_deliq_12mts':0, 'max_deliq_24mts':0, 'max_deliq_6_12mts':0, 
                    'num_times_30p_dpd':0, 'num_times_60p_dpd':0, 'num_times_90p_dpd':0, 'num_times_120p_dpd':0,
                    'Amount Overdue in Last 24 Months':0,'Auto_Closed_Good':0,'Auto_Open':0,'total_medical_accounts':0,'counter_L6M':0,'counter_L7_24M':0,'counter_grt24M':0,'num_TL_nonauth_nonstudent':0,'num_positive_tradelines':0,'positive_MCMC_Trade':0,'negative_MCMC_Trade':0,'Age_Oldest_TL_Mnths_nonauth_nonstu':0,'Age_old_nonaut_max':0,'Age_old_nonaut_min':0,'Age_Oldest_TL_mnths_nonauth_nonstudent':0}
    
    # if 'bureau_xml_data' in input_data['Results'] and 'XPN_Report' in input_data['Results']['bureau_xml_data'] and 'CREDIT_LIABILITY' in input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE'] and len(input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_LIABILITY']) > 0:
    #     if type(input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_LIABILITY']) == list:
    #         data = input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_LIABILITY']
    #     elif type(input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_LIABILITY']) == dict:
    #         data = []
    #         data.append(input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_LIABILITY'])
    # else:
    #     resp = dict.fromkeys(main_rollup, -99999)
    #     resp['Bureau_availability'] = 0
    #     return resp


    main_rollup['Unique_ID'] = oid


    # commented on 27012022

    # try:
    #     main_rollup['total_medical_accounts'] = input_data['Results']['bureau_summary_xml_data']['XPN_Summary_Report']['Collections']['total_medical_accounts'] if 'bureau_xml_data' in input_data['Results'] and 'XPN_Report' in input_data['Results']['bureau_xml_data'] and 'CREDIT_LIABILITY' in input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE'] and len(input_data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_LIABILITY']) > 0 else 0
    # except:
    #     main_rollup['total_medical_accounts'] = 0
#        main_rollup['total_medical_accounts'] = input_data['Results']['bureau_summary_xml_data']['XPN_Summary_Report']['Collections']
 #       if(main_rollup['total_medical_accounts'] in ['',' ',[]]):
  #          main_rollup['total_medical_accounts'] = 0

    #main_rollup['CR_Score_Type'] = input_data['Results']['bureau_summary_xml_data']['XPN_Summary_Report']['Scores']['Scoring']['ScoreModel']['content']
    #main_rollup['CR_Score'] = input_data['Results']['bureau_summary_xml_data']['XPN_Summary_Report']['Scores']['Scoring']['Factor1']['content']
    # added 26012022
    data= app_acc_1
    
    ## Counter Flags variables logic: # modified ....04022022
    data = [{**d,'open_date_new':convert_to_date(d['_AccountOpenedDate']) if '_AccountOpenedDate' in d and d['_AccountOpenedDate'] not in ['',None] else convert_to_date(d['_AccountReportedDate'])} for d in data]
    data = [{**d,'last_act_date':convert_to_date(d['_LastActivityDate']) if '_LastActivityDate' in d and d['_LastActivityDate'] not in ['',None] else convert_to_date(d['_AccountReportedDate'])} for d in data]
    # data = [{**d,'months_diff':(d['last_act_date'] - d['open_date_new']).days /30 if ((d['CreditLoanType'].strip() == 'Auto Loan' and d['_CURRENT_RATING']['_Code'] == 8) or (d['CreditLoanType'].strip() == 'Auto Loan' and d['_CURRENT_RATING']['_Code'] == 9)) else 0} for d in data]
    # data = [{**d,'counter_L6M':1 if (d['months_diff'] >0 and d['months_diff'] <=6) else 0 } for d in data]
    # data = [{**d,'counter_L7_24M':1 if (d['months_diff'] >6 and d['months_diff'] <= 24) else 0 } for d in data]
    # data = [{**d,'counter_grt24M':1 if (d['months_diff'] >24) else 0} for d in data]
    
#     data = [{**d,"recent_pattern": str(d['_PAYMENT_PATTERN']['_Data'][0]) if d['_PAYMENT_PATTERN']['_Data'] not in ['',None] else None} for d in data]
#     data = [{**d,"recent_pattern": str(d['_PAYMENT_PATTERN']['_Data'][0]) if ( (type(d['_PAYMENT_PATTERN']['_Data'])==str) & (d['_PAYMENT_PATTERN']['_Data'] not in ['',None]) ) else str(d['_PAYMENT_PATTERN']['_Data'])} for d in data]    
    data = [{**d,"recent_pattern": '' if '_PAYMENT_PATTERN' not in d or '_Data' not in d['_PAYMENT_PATTERN'] or d['_PAYMENT_PATTERN']['_Data'] in ['',None] else str(d['_PAYMENT_PATTERN']['_Data'])[0]} for d in data]
#     print(data)
    data = [{**d,'months_diff':(d['last_act_date'] - d['open_date_new']).days /30 if(d['CreditLoanType'].strip() == 'Auto Loan') else 0} for d in data]
    
    data = [{**d,'counter_L6M':1 if ((d['months_diff'] >0 and d['months_diff'] <=6 and d['_AccountStatusType'] == 'OPEN' and d['recent_pattern'] in ['3','4','5','6','7','8','9']) or (d['months_diff'] >0 and d['months_diff'] <=6 and d['_AccountStatusType'] == 'CLOSED' and d['recent_pattern'] in ['3','4','5','6','7','8','9'])) else 0} for d in data]
    data = [{**d,'counter_L7_24M':1 if ((d['months_diff'] >6 and d['months_diff'] <= 24 and d['_AccountStatusType'] == 'OPEN' and d['recent_pattern'] in ['3','4','5','6','7','8','9']) or (d['months_diff'] >6 and d['months_diff'] <=24 and d['_AccountStatusType'] == 'CLOSED' and d['recent_pattern'] in ['3','4','5','6','7','8','9'])) else 0} for d in data]
    data = [{**d,'counter_grt24M':1 if ((d['months_diff'] >24 and d['_AccountStatusType'] == 'OPEN' and d['recent_pattern'] in ['3','4','5','6','7','8','9']) or (d['months_diff'] >24 and d['_AccountStatusType'] == 'CLOSED' and d['recent_pattern'] in ['3','4','5','6','7','8','9']))  else 0 } for d in data]


    if data[0]['_AccountType'] == 'Installment':
        main_rollup['first_product'] = 'I'
    elif data[0]['_AccountType'] == 'Revolving':
        main_rollup['first_product'] = 'R'
    else:
        main_rollup['first_product'] = 'X'
        
    if data[-1]['_AccountType'] == 'Installment':
        main_rollup['recent_product'] = 'I'
    elif data[-1]['_AccountType'] == 'Revolving':
        main_rollup['recent_product'] = 'R'
    else:
        main_rollup['recent_product'] = 'X'
        
    main_rollup['Age_Oldest_TL_Mnths'] = round((reqdate - min(convert_to_date(d['_AccountOpenedDate']) if d['_AccountOpenedDate']!='' else convert_to_date(d['_AccountReportedDate']) for d in data)).days / 30)
    main_rollup['Age_Newest_TL_Mnths'] = round((reqdate - max(convert_to_date(d['_AccountOpenedDate']) if d['_AccountOpenedDate']!='' else convert_to_date(d['_AccountReportedDate']) for d in data)).days / 30)

    main_rollup['Age_Oldest_TL_Mnths_nonauth_nonstu'] = [d['_AccountOpenedDate'] if (d['_AccountOpenedDate']!='' and d['CreditLoanType'].strip() !='Education Loan' and d['_AccountOwnershipType']!= 'AuthorizedUser') else d['_AccountReportedDate'] for d in data]

    main_rollup['Age_old_nonaut_max'] = max(main_rollup['Age_Oldest_TL_Mnths_nonauth_nonstu'])
    main_rollup['Age_old_nonaut_min'] = min(main_rollup['Age_Oldest_TL_Mnths_nonauth_nonstu'])
    
    main_rollup['Age_Oldest_TL_mnths_nonauth_nonstudent'] = (reqdate - convert_to_date(main_rollup['Age_old_nonaut_min'])).days/30


    Average_Age_of_Accounts = []
    for d in data:
        open_date = convert_to_date(d['_AccountOpenedDate']) if d['_AccountOpenedDate']!='' else convert_to_date(d['_AccountReportedDate'])    
        account_age = round((reqdate - open_date).days / 30)
        Average_Age_of_Accounts.append(account_age)
        # New Account Adjustments (post 08/11):
        # if not (reqdate >= datetime.strptime('2021-07-01', '%Y-%m-%d') and reqdate <= datetime.strptime('2021-07-31', '%Y-%m-%d') and open_date >= datetime.strptime('2021-08-01', '%Y-%m-%d')):
        main_rollup['Total_TL'] += 1 if (open_date.year>=1900) else 0
        main_rollup['Tot_Active_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED') else 0
        main_rollup['Tot_Closed_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']=='CLOSED') else 0
        main_rollup['Total_TL_Open_L6M'] += 1 if (open_date.year>=1900 and open_date>=(reqdate - relativedelta.relativedelta(months=6))) else 0
        main_rollup['Tot_Active_TL_L6M'] += 1 if (open_date.year>=1900 and open_date>=(reqdate - relativedelta.relativedelta(months=6)) and d['_AccountStatusType']!='CLOSED') else 0
        main_rollup['Total_TL_Open_L12M'] += 1 if (open_date.year>=1900 and open_date>=(reqdate - relativedelta.relativedelta(months=12))) else 0
        main_rollup['Tot_Active_TL_L12M'] += 1 if (open_date.year>=1900 and open_date>=(reqdate - relativedelta.relativedelta(months=12)) and d['_AccountStatusType']!='CLOSED') else 0
        main_rollup['Automobile_TL'] += 1 if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Auto Loan') else 0
        main_rollup['CreditCard_TL'] += 1 if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Credit Card, Terms REV') else 0
        main_rollup['StudentLoan_TL'] += 1 if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Education Loan') else 0
        main_rollup['Secured_TL'] += 1 if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Secured Loan') else 0
        main_rollup['Unsecured_TL'] += 1 if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Unsecured Loan') else 0
        main_rollup['ChargeAccount_TL'] += 1 if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Revolving Charge Account') else 0
        main_rollup['SecuredCreditCard_TL'] += 1 if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Credit Line Secured, Revolving') else 0
        main_rollup['Installment_TL'] += 1 if (open_date.year>=1900 and d['_AccountType']=='Installment') else 0
        main_rollup['Revolving_TL'] += 1 if (open_date.year>=1900 and d['_AccountType']=='Revolving') else 0
        main_rollup['OpenAccount_TL'] = main_rollup['Installment_TL'] + main_rollup['Revolving_TL']
        main_rollup['Num_Open_Automobile_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['CreditLoanType'].strip() =='Auto Loan') else 0
        main_rollup['Num_Open_CreditCard_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['CreditLoanType'].strip() =='Credit Card, Terms REV') else 0
        main_rollup['Num_Open_StudentLoan_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['CreditLoanType'].strip() =='Education Loan') else 0
        main_rollup['Num_Open_Secured_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['CreditLoanType'].strip() =='Secured Loan') else 0
        main_rollup['Num_Open_Unsecured_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['CreditLoanType'].strip() =='Unsecured Loan') else 0
        main_rollup['Num_Open_ChargeAccount_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['CreditLoanType'].strip() =='Revolving Charge Account') else 0
        main_rollup['Num_Open_SecuredCreditCard_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED'and d['CreditLoanType'].strip() =='Credit Line Secured, Revolving') else 0
        main_rollup['Num_Open_Installment_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['_AccountType']=='Installment') else 0
        main_rollup['Num_Open_Revolving_TL'] += 1 if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['_AccountType']=='Revolving') else 0
        main_rollup['Num_Open_OpenAccount_TL'] = main_rollup['Num_Open_Installment_TL'] + main_rollup['Num_Open_Revolving_TL']
        main_rollup['Tot_Automobile_Bal'] += d['_UnpaidBalanceAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Auto Loan') else 0
        main_rollup['Tot_Automobile_Limit'] += d['_HighCreditAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Auto Loan') else 0
        main_rollup['Tot_CreditCard_Bal'] += d['_UnpaidBalanceAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Credit Card, Terms REV') else 0
        main_rollup['Tot_CreditCard_Limit'] += d['_HighCreditAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Credit Card, Terms REV') else 0
        main_rollup['Tot_StudentLoan_Bal'] += d['_UnpaidBalanceAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Education Loan') else 0
        main_rollup['Tot_StudentLoan_Limit'] += d['_HighCreditAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Education Loan') else 0
        main_rollup['Tot_SecuredLoan_Bal'] += d['_UnpaidBalanceAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Secured Loan') else 0
        main_rollup['Tot_SecuredLoan_Limit'] += d['_HighCreditAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Secured Loan') else 0
        main_rollup['Tot_UnsecuredLoan_Bal'] += d['_UnpaidBalanceAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Unsecured Loan') else 0
        main_rollup['Tot_UnsecuredLoan_Limit'] += d['_HighCreditAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Unsecured Loan') else 0
        main_rollup['Tot_ChargeLoan_Bal'] += d['_UnpaidBalanceAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Revolving Charge Account') else 0
        main_rollup['Tot_ChargeLoan_Limit'] += d['_HighCreditAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Revolving Charge Account') else 0
        main_rollup['Tot_SecuredCreditCard_Bal'] += d['_UnpaidBalanceAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Credit Line Secured, Revolving') else 0
        main_rollup['Tot_SecuredCreditCard_Limit'] += d['_HighCreditAmount'] if (open_date.year>=1900 and d['CreditLoanType'].strip() =='Credit Line Secured, Revolving') else 0            
        main_rollup['Tot_Installment_Bal'] += d['_UnpaidBalanceAmount'] if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['_AccountType']=='Installment') else 0
        main_rollup['Tot_Installment_Limit'] += d['_HighCreditAmount'] if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['_AccountType']=='Installment') else 0
        main_rollup['Tot_Revolving_Limit'] += d['_HighCreditAmount'] if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['_AccountType']=='Revolving') else 0
        main_rollup['Tot_Revolving_Bal'] += d['_UnpaidBalanceAmount'] if (open_date.year>=1900 and d['_AccountStatusType']!='CLOSED' and d['_AccountType']=='Revolving') else 0
        main_rollup['num_TL_nonauth_nonstudent'] += 1 if(d['_AccountOwnershipType']!= 'AuthorizedUser' and d['CreditLoanType'].strip() != 'Education Loan') else 0 
        # main_rollup['num_positive_tradelines'] += 1 if(d['_AccountOwnershipType']!= 'AuthorizedUser' and d['CreditLoanType'].strip() != 'Education Loan' and d['_CURRENT_RATING']['_Code'] == 'C') else 0
        # main_rollup['positive_MCMC_Trade'] += 1 if(d['_AccountOwnershipType']!= 'AuthorizedUser' and d['CreditLoanType'].strip() != 'Education Loan' and d['_CREDITOR']['_Name'] =='MIKE CARLSON MOTOR CO' and d['_CURRENT_RATING']['_Code'] == 'C') else 0
        # main_rollup['negative_MCMC_Trade'] += 1 if(d['_AccountOwnershipType']!= 'AuthorizedUser' and d['CreditLoanType'].strip() != 'Education Loan' and d['_CREDITOR']['_Name'] =='MIKE CARLSON MOTOR CO' and d['_CURRENT_RATING']['_Code'] != 'C') else 0
        main_rollup['num_positive_tradelines'] += 1 if(d['_AccountOwnershipType']!= 'AuthorizedUser' and d['CreditLoanType'].strip() != 'Education Loan' and d['recent_pattern'] in ['C','X','N']) else 0
        main_rollup['positive_MCMC_Trade'] += 1 if(d['_AccountOwnershipType']!= 'AuthorizedUser' and d['CreditLoanType'].strip() != 'Education Loan' and d['_CREDITOR']['_Name'] =='MIKE CARLSON MOTOR CO' and d['recent_pattern'] in ['C','X','N'] ) else 0
        main_rollup['negative_MCMC_Trade'] += 1 if(d['_AccountOwnershipType']!= 'AuthorizedUser' and d['CreditLoanType'].strip() != 'Education Loan' and d['_CREDITOR']['_Name'] =='MIKE CARLSON MOTOR CO' and d['recent_pattern'] not in ['C','X','N'] ) else 0
        
        main_rollup['Auto_Open'] += 0 if (('CreditLoanType' not in d) or 
                                          ('_AccountStatusType' not in d) ) else 1 if ( (d['CreditLoanType'].strip() =='Auto Loan') and 
                                          (d['_AccountStatusType'] == 'OPEN') and (d['recent_pattern'] in ['C','X','N','1','2']) ) else 0
        # auto open recent pattern 
        main_rollup['Auto_Closed_Good'] +=0 if ( ('CreditLoanType' not in d) or 
                                                ('_AccountStatusType' not in d) or 
                                                ('_UnpaidBalanceAmount' not in d) or 
                                                ('_PAYMENT_PATTERN' not in d) or 
                                                ('_Data' not in d['_PAYMENT_PATTERN']) ) else 1 if ( (d['CreditLoanType'].strip() =='Auto Loan') and 
                                                (d['_AccountStatusType']=='CLOSED') and 
                                                (str(d['_PAYMENT_PATTERN']['_Data'])[0] not in ['3','4','5','6','7','8','9']) and 
                                                (d['_UnpaidBalanceAmount']==0) ) else 0

        if account_age<=24:
            main_rollup['Amount Overdue in Last 24 Months'] += d['_PastDueAmount'] 
        
    main_rollup['Tot_OpenAccount_Bal'] = main_rollup['Tot_Installment_Bal'] + main_rollup['Tot_Revolving_Bal']
    main_rollup['Tot_OpenAccount_Limit'] = main_rollup['Tot_Installment_Limit'] + main_rollup['Tot_Revolving_Limit']

    main_rollup['Average Age of Accounts'] = int(np.mean(Average_Age_of_Accounts))

    
    # data = [{**d, 'split_paygrid': split_grid(d['_PAYMENT_PATTERN']['_Data'])} for d in data]
    # data = [{**d, 'split_paygrid': ['O' if item in ['*'] else (item) for item in d['split_paygrid']]} for d in data]
    # data = [{**d, 'split_paygrid': ['O' if item in ['/'] else (item) for item in d['split_paygrid']]} for d in data]
    # data = [{**d, 'split_paygrid': [-1 if item in ['X','O','Z','C','N'] else int(item) for item in d['split_paygrid']]} for d in data]
    
    # offset part
    data = [{**d, 'grid': str(d['_PAYMENT_PATTERN']['_Data'])} for d in data]
    data = [{**d, 'PAYMENTHISTORY_ext_strngs': "Z" * (12 - len(d['grid']))} for d in data]
    data = [{**d, 'PAYMENTHISTORY_full_strngs': d['PAYMENTHISTORY_ext_strngs'] + str(d['grid'])} for d in data]
    data = [{**d, 'CreatedDate_new':reqDate} for d in data]
    data = [{**d, 'payment_startdate': convert_to_date(d['_PAYMENT_PATTERN']['_StartDate'])} for d in data]  
    data = [{**d, 'mon_diff_offset': abs(round((d['CreatedDate_new'] - d['payment_startdate']).days /30)) } for d in data]
    data = [{**d, 'offset_strings': "O" * int(d['mon_diff_offset'])} for d in data]
    data = [{**d, 'PAYMENTGRID_with_offset': d['offset_strings'] + d['PAYMENTHISTORY_full_strngs']} for d in data]
    # modified ... 03022022 
    data = [{**d, 'split_paygrid_full':d['PAYMENTGRID_with_offset'][:12]} for d in data]  
    data = [{**d, 'split_paygrid_open_test': d['split_paygrid_full'] if d['_AccountStatusType']!='CLOSED' else '' }for d in data]
    data = [{**d, 'split_paygrid_open': open_grid(d['split_paygrid_open_test'])} for d in data]
    data = [{**d, 'split_paygrid_test': [-1 if item not in ['1','2','3','4','5','6','7','8','9'] else int(item) for item in d['split_paygrid_full']]} for d in data]
    data = [{**d, 'split_paygrid':d['split_paygrid_test']} for d in data]
    data = [{**d, 'rvrs_split_paygrid': list(reversed(d['split_paygrid']))} for d in data]
    data = [{**d, 'recent_deliq': next((j for i, j in enumerate(d['split_paygrid']) if j > 0 and j is not None), None)} for d in data]
    data = [{**d, 'old_dpd_ind': (24 - next((i for i, j in enumerate(d['rvrs_split_paygrid']) if j > 0 and j is not None), None)) if next((i for i, j in enumerate(d['rvrs_split_paygrid']) if j > 0 and j is not None), None) != None else None,
                  'recnt_dpd_ind': next((i for i, j in enumerate(d['split_paygrid']) if j > 0 and j is not None), None),
                  'filt_gt_1': list(filter(lambda x: (int(x) > 1), d['split_paygrid']))} for d in data]         
               
        
    data = [{**d, 'maxdpdval': max(d['filt_gt_1']) if len(d['filt_gt_1']) > 1 else None,
                  'mindpdval': min(d['filt_gt_1']) if len(d['filt_gt_1']) > 1 else None,
                  'time_since_recnt_deliq': (d['recnt_dpd_ind'] + 1)  if d['recnt_dpd_ind'] is not None else -99999,
                  'time_since_oldest_deliq': (d['old_dpd_ind']) if d['old_dpd_ind'] is not None else None,
                  'num_times_deliq': get_split_paygrid_res(d['split_paygrid'], 0, None, None, 'count', False),
                  'num_times_deliq_open':get_split_paygrid_res(d['split_paygrid_open'], 0, None, None, 'count', False),
                  'num_times_30p': get_split_paygrid_res(d['split_paygrid'], 1, None, None, 'count', 'yes'),
                  'num_times_60p': get_split_paygrid_res(d['split_paygrid'], 2, None, None, 'count', 'yes'),
                  'num_times_90p': get_split_paygrid_res(d['split_paygrid'], 3, None, None, 'count', 'yes'),
                  'num_times_120p': get_split_paygrid_res(d['split_paygrid'], 4, None, None, 'count', 'yes'),
                  'num_deliq_6mts': get_split_paygrid_res(d['split_paygrid'], 1, None, 6, 'count', False),
                  'num_deliq_12mts': get_split_paygrid_res(d['split_paygrid'], 1, None, 12, 'count', False),
                  'num_deliq_24mts': get_split_paygrid_res(d['split_paygrid'], 1, None, 24, 'count', False),
                  'num_deliq_36mts': get_split_paygrid_res(d['split_paygrid'], 1, None, 36, 'count', False),
                  'num_deliq_48mts': get_split_paygrid_res(d['split_paygrid'], 1, None, 48, 'count', False),
                  'num_deliq_6_12mts': get_split_paygrid_res(d['split_paygrid'], 1, 6, 12, 'count', False),
                  'max_deliq_6mts': get_split_paygrid_res(d['split_paygrid'], 1, 1, 6, 'max', True),
                  'max_deliq_12mts': get_split_paygrid_res(d['split_paygrid'], 1, None, 12, 'max', True),
                  'max_deliq_24mts': get_split_paygrid_res(d['split_paygrid'], 1, None, 24, 'max', True),
                  'max_deliq_36mts': get_split_paygrid_res(d['split_paygrid'], 1, None, 36, 'max', True),
                  'max_deliq_48mts': get_split_paygrid_res(d['split_paygrid'], 1, None, 48, 'max', True),
                  'max_deliq_6_12mts': get_split_paygrid_res(d['split_paygrid'], 1, 6, 12, 'max', True)
                  } for d in data]
    
    #logger.info([(d['max_deliq_6mts']) for d in data])
    new_acc = [{'time_since_first_deliquency': d['time_since_oldest_deliq'], 'time_since_recent_deliquency': d['time_since_recnt_deliq'],
                'num_times_delinquent': d['num_times_deliq'], 'max_delinquency_level': d['maxdpdval'],
                'num_deliq_6mts': d['num_deliq_6mts'], 'num_deliq_12mts': d['num_deliq_12mts'],
                'num_deliq_24mts': d['num_deliq_24mts'], 'num_deliq_36mts': d['num_deliq_36mts'], 'num_deliq_48mts': d['num_deliq_48mts'],
                'num_deliq_6_12mts': d['num_deliq_6_12mts'], 'max_deliq_6mts': d['max_deliq_6mts'],
                'max_deliq_12mts': d['max_deliq_12mts'], 'max_deliq_24mts': d['max_deliq_24mts'], 'max_deliq_36mts': d['max_deliq_36mts'],
                'max_deliq_48mts': d['max_deliq_48mts'], 'max_deliq_6_12mts': d['max_deliq_6_12mts'],
                'num_times_30p_dpd': d['num_times_30p'], 'num_times_60p_dpd': d['num_times_60p'], 'num_times_90p_dpd': d['num_times_90p'],
                'num_times_120p_dpd': d['num_times_120p'], 'recent_deliq': d['recent_deliq'],
                'max_recent_deliq': d['recent_deliq'], 'num_tradelines': 1} for d in data]    
    
    sorted_acc = sorted(new_acc, key=lambda i: (i['time_since_recent_deliquency'], i['recent_deliq']))
    if len(sorted_acc) > 0:
        sorted_acc = sorted_acc[0]
        sorted_acc['time_since_recent_deliquency'] = None if sorted_acc['time_since_recent_deliquency'] == -99999 else sorted_acc['time_since_recent_deliquency']
    else:
        sorted_acc = {}
        sorted_acc['time_since_recent_deliquency'] = None
        
    main_rollup['time_since_first_deliquency'] = excl_none_operation([d['time_since_first_deliquency'] for d in new_acc], 'max')
    main_rollup['time_since_recent_deliquency'] = excl_none_operation([d['time_since_recent_deliquency'] for d in new_acc], 'min')
    main_rollup['num_times_delinquent'] = excl_none_operation([d['num_times_delinquent'] for d in new_acc], 'sum')
    main_rollup['num_delinquent_TL'] = excl_none_operation([d['num_times_deliq_open'] for d in data], 'count_non_zero')
    main_rollup['max_delinquency_level'] = excl_none_operation([d['max_delinquency_level'] for d in new_acc], 'max')
    main_rollup['max_recent_deliq'] = excl_none_operation([d['max_recent_deliq'] for d in new_acc], 'max')
    main_rollup['recent_deliq'] = new_acc[0]['recent_deliq'] if (len(new_acc)>0 and new_acc[0]['recent_deliq']) else -99999
    main_rollup['num_deliq_6mts'] = excl_none_operation([d['num_deliq_6mts'] for d in new_acc], 'sum')
    main_rollup['num_deliq_12mts'] = excl_none_operation([d['num_deliq_12mts'] for d in new_acc], 'sum')
    main_rollup['num_deliq_24mts'] = excl_none_operation([d['num_deliq_24mts'] for d in new_acc], 'sum')
    main_rollup['num_deliq_6_12mts'] = excl_none_operation([d['num_deliq_6_12mts'] for d in new_acc], 'sum')
    main_rollup['num_tradelines'] = excl_none_operation([d['num_tradelines'] for d in new_acc], 'sum')
    main_rollup['max_deliq_6mts'] = excl_none_operation([d['max_deliq_6mts'] for d in new_acc], 'max')
    main_rollup['max_deliq_12mts'] = excl_none_operation([d['max_deliq_12mts'] for d in new_acc], 'max')
    main_rollup['max_deliq_24mts'] = excl_none_operation([d['max_deliq_24mts'] for d in new_acc], 'max')
    main_rollup['max_deliq_6_12mts'] = excl_none_operation([d['max_deliq_6_12mts'] for d in new_acc], 'max')
    main_rollup['num_times_30p_dpd'] = excl_none_operation([d['num_times_30p_dpd'] for d in new_acc], 'sum')
    main_rollup['num_times_60p_dpd'] = excl_none_operation([d['num_times_60p_dpd'] for d in new_acc], 'sum')
    main_rollup['num_60dpd_TL'] = excl_none_operation([d['num_times_60p_dpd'] for d in new_acc], 'count_non_zero')
    main_rollup['num_times_90p_dpd'] = excl_none_operation([d['num_times_90p_dpd'] for d in new_acc], 'sum')
    main_rollup['num_times_120p_dpd'] = excl_none_operation([d['num_times_120p_dpd'] for d in new_acc], 'sum')
    main_rollup['counter_L6M'] = sum(d['counter_L6M'] for d in data)
    main_rollup['counter_L7_24M'] = sum(d['counter_L7_24M'] for d in data)
    main_rollup['counter_grt24M'] = sum(d['counter_grt24M'] for d in data)

    main_rollup = {**main_rollup, **rollup_bankruptcy(app_sum_1, reqdate), **rollup_collection(app_sum_1, reqdate), **rollup_inquiry(app_inq_1, reqdate),'Bureau_availability': 1}  

    main_rollup = {**main_rollup,
                   'pct_of_active_TLs_ever': round(main_rollup['Tot_Active_TL']/main_rollup['Total_TL'],3) if (main_rollup['Tot_Active_TL']>=0 and main_rollup['Total_TL']>0) or (main_rollup['Tot_Active_TL']<0 and main_rollup['Total_TL']<0) else 0,
                   'pct_enq_L6m_of_L12m': round(main_rollup['enq_L6m']/main_rollup['enq_L12m'],3) if (main_rollup['enq_L6m']>=0 and main_rollup['enq_L12m']>0) or (main_rollup['enq_L6m']<0 and main_rollup['enq_L12m']<0) else 0,
                   'pct_opened_TLs_L6m_of_L12m': round(main_rollup['Total_TL_Open_L6M']/main_rollup['Total_TL_Open_L12M'],3) if (main_rollup['Total_TL_Open_L6M']>=0 and main_rollup['Total_TL_Open_L12M']>0) or (main_rollup['Total_TL_Open_L6M']<0 and main_rollup['Total_TL_Open_L12M']<0) else 0,
                   'Automobile_cuurbal_highcredit': round((main_rollup['Tot_Automobile_Bal']/main_rollup['Tot_Automobile_Limit']),3) if (main_rollup['Tot_Automobile_Bal']>0) and (main_rollup['Tot_Automobile_Limit']>0) else -99999,
                   'util': 100*round((main_rollup['Tot_CreditCard_Bal']/main_rollup['Tot_CreditCard_Limit']),4) if main_rollup['Tot_CreditCard_Limit']>0 else -99999,
                   'StudentLoan_cuurbal_highcredit': round((main_rollup['Tot_StudentLoan_Bal']/main_rollup['Tot_StudentLoan_Limit']),3) if (main_rollup['Tot_StudentLoan_Bal']>0) and (main_rollup['Tot_StudentLoan_Limit']>0) else -99999,
                   'ChargeAccount_cuurbal_highcredit': round((main_rollup['Tot_ChargeLoan_Bal']/main_rollup['Tot_ChargeLoan_Limit']),3) if (main_rollup['Tot_ChargeLoan_Bal']>0) and (main_rollup['Tot_ChargeLoan_Limit']>0) else -99999,
                   'Mortgage_cuurbal_highcredit': round((main_rollup['Tot_Mortgage_Bal']/main_rollup['Tot_Mortgage_Limit']),3) if (main_rollup['Tot_Mortgage_Bal']>0) and (main_rollup['Tot_Mortgage_Limit']>0) else -99999,
                   'Secured_cuurbal_highcredit': round((main_rollup['Tot_SecuredLoan_Bal']/main_rollup['Tot_SecuredLoan_Limit']),3) if (main_rollup['Tot_SecuredLoan_Bal']>0) and (main_rollup['Tot_SecuredLoan_Limit']>0) else -99999,
                   'Unsecured_cuurbal_highcredit': round((main_rollup['Tot_UnsecuredLoan_Bal']/main_rollup['Tot_UnsecuredLoan_Limit']),3) if (main_rollup['Tot_UnsecuredLoan_Bal']>0) and (main_rollup['Tot_UnsecuredLoan_Limit']>0) else -99999,
                   'Intallment_cuurbal_highcredit':round((main_rollup['Tot_Installment_Bal']/main_rollup['Tot_Installment_Limit']),3) if (main_rollup['Tot_Installment_Bal']>0) and (main_rollup['Tot_Installment_Limit']>0) else -99999,
                   'Revolving_cuurbal_highcredit':round((main_rollup['Tot_Revolving_Bal']/main_rollup['Tot_Revolving_Limit']),3) if (main_rollup['Tot_Revolving_Bal']>0) and (main_rollup['Tot_Revolving_Limit']>0) else -99999,
                   'SecuredCreditCard_cuurbal_highcredit': round((main_rollup['Tot_SecuredCreditCard_Bal']/main_rollup['Tot_SecuredCreditCard_Limit']),3) if (main_rollup['Tot_SecuredCreditCard_Bal']>0) and (main_rollup['Tot_SecuredCreditCard_Limit']>0) else -99999,
                   'Total_cur_blnc_to_Original_amount': 100*round((main_rollup['Tot_OpenAccount_Bal']/main_rollup['Tot_OpenAccount_Limit']),4) if main_rollup['Tot_OpenAccount_Limit']>0 else -99999,
                   'Automobile_Flag': 1 if main_rollup['Automobile_TL'] > 0 else 0,
                   'ChargeAccount_Flag': 1 if main_rollup['ChargeAccount_TL'] > 0 else 0,
                   'CreditCard_Flag': 1 if main_rollup['CreditCard_TL'] > 0 else 0,
                   'Installment_Flag': 1 if main_rollup['Installment_TL'] > 0 else 0,
                   'Mortgage_Flag': 1 if main_rollup['Mortgage_TL'] > 0 else 0,
                   'OpenAccount_Flag': 1 if main_rollup['OpenAccount_TL'] > 0 else 0,
                   'SecuredCreditCard_Flag': 1 if main_rollup['SecuredCreditCard_TL'] > 0 else 0,
                   'Secured_Flag': 1 if main_rollup['Secured_TL'] > 0 else 0,
                   'StudentLoan_Flag': 1 if main_rollup['StudentLoan_TL'] > 0 else 0,
                   'Unsecured_Flag': 1 if main_rollup['Unsecured_TL'] > 0 else 0,
                   'PL_Flag': 1 if main_rollup['PersonalLoan_TL'] > 0 else 0,
                   'Revolving_Flag': 1 if main_rollup['Revolving_TL'] > 0 else 0,
                   'pct_tot_deliq_TLs': 100*round(main_rollup['num_delinquent_TL']/main_rollup['Total_TL'],4) if main_rollup['Total_TL']>0 else -99999,
                   'satisfactory_pct': 100 - 100*round(main_rollup['num_delinquent_TL']/main_rollup['Total_TL'],4) if main_rollup['Total_TL']>0 else -99999,
                   'pct_enq_L6m_of_ever': round(main_rollup['enq_L6m']/main_rollup['tot_enq'],3) if (main_rollup['enq_L6m']>=0 and main_rollup['tot_enq']>0) or (main_rollup['enq_L6m']<0 and main_rollup['tot_enq']<0) else 0,
                   'delinquent_flag_Yes': 1 if main_rollup['num_times_delinquent']>0 else 0,
                   'delinquent_flag_No': 1 if main_rollup['num_times_delinquent']<=0 else 0,
                   'enql1/l3': round((main_rollup['enq_L1m']/main_rollup['enq_L3m']),3) if  ( (main_rollup['enq_L1m']>=0)  and(main_rollup['enq_L3m']>0)) else -99999,
                   'enql3/l6': round((main_rollup['enq_L3m']/main_rollup['enq_L6m']),3) if ((main_rollup['enq_L3m']>=0) and (main_rollup['enq_L6m']>0)) else -99999 ,
                   'enql3/l12': round((main_rollup['enq_L3m']/main_rollup['enq_L12m']),3) if ((main_rollup['enq_L3m']>=0) and (main_rollup['enq_L12m']>0)) else -99999,
                   'enql6/l12': round((main_rollup['enq_L6m']/main_rollup['enq_L12m']),3) if ((main_rollup['enq_L6m']>=0) and (main_rollup['enq_L12m']>0)) else -99999,
                   'Ratio_opentrd_6M_12M': round((main_rollup['Total_TL_Open_L6M']/main_rollup['Total_TL_Open_L12M']),3) if (main_rollup['Total_TL_Open_L6M']>=0) and (main_rollup['Total_TL_Open_L12M']>0) else -99999,
                   'Enq_conversion_L6M': round((main_rollup['Total_TL_Open_L6M']/main_rollup['enq_L6m']),3) if (main_rollup['Total_TL_Open_L6M']>=0)and (main_rollup['enq_L6m']>0) else -99999,
                   'Enq_conversion_L12M': round((main_rollup['Total_TL_Open_L12M']/main_rollup['enq_L12m']),3) if (main_rollup['Total_TL_Open_L12M']>=0) and (main_rollup['enq_L12m']>0) else -99999,
                   'first_product_O': 'Yes' if main_rollup['first_product'] == 'O' else 'No',
                   'first_product_I': 'Yes' if main_rollup['first_product'] == 'I' else 'No',
                   'first_product_R': 'Yes' if main_rollup['first_product'] == 'R' else 'No',
                   'first_product_others': 'Yes' if main_rollup['first_product'] == 'others' else 'No',
                   'recent_product_O': 'Yes' if main_rollup['recent_product'] == 'O' else 'No',
                   'recent_product_I': 'Yes' if main_rollup['recent_product'] == 'I' else 'No',
                   'recent_product_R': 'Yes' if main_rollup['recent_product'] == 'R' else 'No',
                   'recent_product_others': 'Yes' if main_rollup['recent_product'] == 'others' else 'No',
                   'CreatedDate_new':reqdate.strftime('%d-%m-%Y %H:%M:%S'),
                   'Non_medical_collections_count_ever': main_rollup['times_collections_ever_count'] - main_rollup['total_medical_accounts']
                   }
    main_rollup.pop('first_product')
    main_rollup.pop('recent_product')
    new_rollup_var = dict()
    for k, v in main_rollup.items():
        if k in {'Auto_Closed_Good','Auto_Open','error_status','Unique_ID','Total_cur_blnc_to_Original_amount','Average Age of Accounts','first_product_R','Total_TL','num_delinquent_TL','Tot_Closed_TL','Tot_Revolving_Bal','Unsecured_TL','recent_deliq','enql1/l3','Num_Open_CreditCard_TL','Amount Overdue in Last 24 Months','util','time_since_recent_deliquency','satisfactory_pct','num_times_90p_dpd','num_60dpd_TL','enq_L12m','enq_L6m','Age_Oldest_TL_Mnths','counter_L6M','counter_L7_24M','counter_grt24M','num_TL_nonauth_nonstudent','num_positive_tradelines','negative_MCMC_Trade','positive_MCMC_Trade','times_collections_ever_count','total_medical_accounts','Non_medical_collections_count_ever','Bureau_availability','Age_Oldest_TL_mnths_nonauth_nonstudent'}:
            new_rollup_var[k] = v
    return(new_rollup_var)


In [68]:
__author__ = 'Kushal'

import json
# from app.main import logger
from datetime import datetime, date, timedelta
from dateutil import relativedelta
import requests
import numpy as np
# from app.main.model_validation.rollups.equifax_rollup.utils import rollup_utils
# from app.main.decorators.exception_decorator import handle_exceptions



# @handle_exceptions
def rollup_variables(data):
    
    try:
    
        invalid_list =['',None,[],{},[{}]," ",' ']
        list_vars={'Auto_Closed_Good','Auto_Open','Unique_ID','Total_cur_blnc_to_Original_amount','Average Age of Accounts','first_product_R','Total_TL','num_delinquent_TL','Tot_Closed_TL','Tot_Revolving_Bal','Unsecured_TL','recent_deliq','enql1/l3','Num_Open_CreditCard_TL','Amount Overdue in Last 24 Months','util','time_since_recent_deliquency','satisfactory_pct','num_times_90p_dpd','num_60dpd_TL','enq_L12m','enq_L6m','Age_Oldest_TL_Mnths', 'Bureau_availability','counter_L6M','counter_L7_24M','counter_grt24M','num_TL_nonauth_nonstudent','num_positive_tradelines','negative_MCMC_Trade','positive_MCMC_Trade','times_collections_ever_count','total_medical_accounts','Non_medical_collections_count_ever','Age_Oldest_TL_mnths_nonauth_nonstudent','error_status'}

        if(data not in invalid_list):

            if('bureau_xml_data' in data['Results'] and data['Results']['bureau_xml_data'] not in invalid_list ):
                if('XPN_Report' in data['Results']['bureau_xml_data'] and data['Results']['bureau_xml_data']['XPN_Report'] not in invalid_list and 'CREDIT_RESPONSE' in data['Results']['bureau_xml_data']['XPN_Report'] and data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE'] not in invalid_list):
                    borrower= data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['BORROWER']
                    bor_list =['BorRec0001','BorRec0002']

                    invalid_list =[None,'',""," ",[],{},[{}]]
                    app_acc_1 = []
                    app_acc_2 = []
                    app_inq_1 = []
                    app_inq_2 = []
                    app_sum_1 = []


                    oid=data['Results']['Transaction_Information']['Transid']
                    reqDate = datetime.now()
                    reqdate = datetime.strptime(data['submitted_datetime'][0:10], '%Y-%m-%d') if 'submitted_datetime' in data else datetime.now() 
                    acc_list =data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_LIABILITY']
                    inq_list =data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_INQUIRY']
                    summ_list = data['Results']['bureau_xml_data']['XPN_Report']['CREDIT_RESPONSE']['CREDIT_SUMMARY']['_DATA_SET']

                    try:
                        if(data['Results']['bureau_summary_xml_data']['XPN_Summary_Report'] not in invalid_list):
                            bur_list = data['Results']['bureau_summary_xml_data']['XPN_Summary_Report']
                        else:
                            bur_list = None
                    except:
                        bur_list = None


                    if(bur_list not in invalid_list):
                        for i in bur_list:
                            if(len(bur_list)>1):
                                try:
                                    bur_list_1=(bur_list[0]['Collections']['total_medical_accounts'])
                                except:
                                    bur_list_1 =0

                                try:
                                    bur_list_2=(bur_list[1]['Collections']['total_medical_accounts'])
                                except:
                                    bur_list_2 = 0
                            elif(len(bur_list) == 1):
                                try:
                                    bur_list_1=(bur_list[0]['Collections']['total_medical_accounts'])
                                except:
                                    bur_list_1 = 0
                            else:
                                bur_list_1 = 0
                    else:
                        bur_list_1= 0

                    if(acc_list not in invalid_list):
                        if type(acc_list)==list:
                            for i in acc_list:
                                if(i['BorrowerID'] == bor_list[0]):
                                    app_acc_1.append(i)
                                elif(i['BorrowerID'] == bor_list[1]):
                                    app_acc_2.append(i)
                                else:
                                    pass
                        else:
                            if acc_list['BorrowerID'] == bor_list[0]:
                                app_acc_1.append(acc_list)
                            elif acc_list['BorrowerID'] == bor_list[1]:
                                app_acc_2.append(acc_list)
                            else:
                                pass
                    else:
                        pass

                    if(inq_list not in invalid_list):
                        for i in inq_list:
                            if(i['BorrowerID'] == bor_list[0]):
                                app_inq_1.append(i)
                            elif(i['BorrowerID'] == bor_list[1]):
                                app_inq_2.append(i)

                            else:
                                pass
                    else:
                        pass

                    if(summ_list not in invalid_list):
                        for i in summ_list:
                            app_sum_1.append(i)
                    else:
                        pass

                    res1a={}
                    res1b={}
                    rollup_vars = []
                    if(app_acc_1 and app_inq_1 and app_sum_1 not in [None,[],{}]):
                        try:
                            res1a=(mcmc_rollup(oid, app_acc_1,app_inq_1,app_sum_1,bur_list_1,reqdate))
                        except:
                            res1a ={}
                            res1a=res(list_vars,res1a)
                    else:
                        res1a = {}
                        res1a=res(list_vars,res1a)
                    if(app_acc_2 and app_inq_2 and app_sum_1 not in [None,[],{}],):
                        try:
                            res1b=(mcmc_rollup(oid, app_acc_2,app_inq_2,app_sum_1,bur_list_2,reqdate))
                        except:
                            res1b = {}   
                            res1b=res(list_vars,res1b)
                    else:
                        res1b = {}
                        res1b=res(list_vars,res1b)
                    rollup_vars.append(res1a)
                    rollup_vars.append(res1b)

                    return rollup_vars
                else:
                    res1a ={}
                    res1a=res(list_vars,res1a)
                    res1b ={}
                    res1b=res(list_vars,res1b)
                    rollup_vars =[]
                    rollup_vars.append(res1a)
                    rollup_vars.append(res1b)
                    return rollup_vars

            else:
                res1a ={}
                res1a=res(list_vars,res1a)
                res1b ={}
                res1b=res(list_vars,res1b)
                rollup_vars =[]
                rollup_vars.append(res1a)
                rollup_vars.append(res1b)
                return rollup_vars


        else:
            res1a ={}
            res1a=res(list_vars,res1a)
            res1b ={}
            res1b=res(list_vars,res1b)
            rollup_vars =[]
            rollup_vars.append(res1a)
            rollup_vars.append(res1b)
            return rollup_vars
    except:

        res1a ={}
        res1a=res(list_vars,res1a)
        res1b ={}
        res1b=res(list_vars,res1b)
        rollup_vars =[]
        rollup_vars.append(res1a)
        rollup_vars.append(res1b)
        
        return rollup_vars

            
        






In [69]:
import json
from datetime import datetime

data= json.load(open('f9759670-974c-4ea6-b94d-10121570b485-Full_CounterNull.json'))
reqDate= datetime.now()
rollup_variables(data['values']['LN_EXP_APP']['values']['700Credit'])

[{'Unique_ID': '700D081512846',
  'error_status': 'Bureau pull Success',
  'Unsecured_TL': 0,
  'Num_Open_CreditCard_TL': 0,
  'Total_TL': 1,
  'Tot_Closed_TL': 1,
  'Average Age of Accounts': 5,
  'Age_Oldest_TL_Mnths': 5,
  'Tot_Revolving_Bal': 0,
  'time_since_recent_deliquency': 9,
  'Bureau_availability': 1,
  'recent_deliq': 9,
  'num_times_90p_dpd': 0,
  'Amount Overdue in Last 24 Months': 1018,
  'Auto_Closed_Good': 0,
  'Auto_Open': 0,
  'total_medical_accounts': 0,
  'counter_L6M': 0,
  'counter_L7_24M': 0,
  'counter_grt24M': 0,
  'num_TL_nonauth_nonstudent': 1,
  'num_positive_tradelines': 0,
  'positive_MCMC_Trade': 0,
  'negative_MCMC_Trade': 0,
  'Age_Oldest_TL_mnths_nonauth_nonstudent': 5.4,
  'num_delinquent_TL': 0,
  'num_60dpd_TL': 0,
  'times_collections_ever_count': 1,
  'enq_L6m': 6,
  'enq_L12m': 6,
  'util': -99999,
  'Total_cur_blnc_to_Original_amount': -99999,
  'satisfactory_pct': 100.0,
  'enql1/l3': 1.0,
  'first_product_R': 'No',
  'Non_medical_collections